In [34]:
import os, sys
sys.path.insert(1, os.path.join(os.getcwd()  , '../..'))

In [35]:
# imports
import random
import multiprocessing as mp
from collections import defaultdict
from core import * 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [46]:
possible_items = [
    TwoDimensionalItem(300, 400),
    TwoDimensionalItem(600, 300),
    TwoDimensionalItem(600, 400),
    TwoDimensionalItem(300, 200),
]

vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600),
    ]
)

# Define some problems

# Problem 1
nb_items_1 = 15
V_1 = list(range(0, nb_items_1 * 2 + 1))
random.seed(0)
items_1 = random.choices(possible_items, k=nb_items_1)
random.seed(0)
distance_matrix_1 = [
    [random.randint(1, 100) if i != 0 and j!=0 and i != j else 0 for i in V_1]
    for j in V_1
]
problem_1 = TwoDimensionalProblem(items_1, vehicle, distance_matrix_1)

# Problem 2
nb_items_2 = 15
V_2 = list(range(0, nb_items_2 * 2 + 1))
random.seed(100)
items_2 = random.choices(possible_items, k=nb_items_2)
random.seed(100)
distance_matrix_2 = [
    [random.randint(1, 100) if i != 0 and j!=0 and i != j else 0 for i in V_2]
    for j in V_2
]
problem_2 = TwoDimensionalProblem(items_2, vehicle, distance_matrix_2)

# %%
# Initial solutions for the problem by running the gurobi solver
initial_solution_time_limit = 2 # 1 second

# Problem 1
modelled_problem_1 = ModelledTwoDimensionalProblem(items_1, vehicle)
modelled_problem_1.create_model()
modelled_problem_1.apply_constraints()
modelled_problem_1.set_model_objective()
modelled_problem_1.solve(time_limit=initial_solution_time_limit)
initial_solution_1 = modelled_problem_1.extract_solution()

# Problem 2
modelled_problem_2 = ModelledTwoDimensionalProblem(items_2, vehicle)
modelled_problem_2.create_model()
modelled_problem_2.apply_constraints()
modelled_problem_2.set_model_objective()
modelled_problem_2.solve(time_limit=initial_solution_time_limit)
initial_solution_2 = modelled_problem_2.extract_solution()

initial_solutions_dict = {
    problem_1: initial_solution_1,
    problem_2: initial_solution_2,
}

Set parameter TimeLimit to value 2
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 49029 rows, 4724 columns and 239949 nonzeros
Model fingerprint: 0x36e36bfe
Variable types: 0 continuous, 4724 integer (4297 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 8e+02]
Presolve removed 2384 rows and 584 columns
Presolve time: 1.55s
Presolved: 46645 rows, 4140 columns, 176028 nonzeros
Variable types: 0 continuous, 4140 integer (3720 binary)
Found heuristic solution: objective 3242.0000000

Root relaxation: time limit, 0 iterations, 0.14 seconds (0.09 work units)

Explored 1 nodes (0 simplex iterations) in 2.01 seconds (1.10 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 3242 

Time limit reached
Best objective 3.242000000000e+03, best bound 0.0

In [147]:
solver_single = LNS(
    problem_1,
    initial_solution_1,
    time_limit=200,
    # nb_processes=11,
    min_destruction_degree=0.05,
    max_destruction_degree=0.2,
    # destroy_strategy=DestroyStrategy.HIGHEST_COST,
    # repair_strategy=RepairStrategy.GREEDY_LEAST_COST_INSERT,
)
# solver.set_destruction_degree_criterion(DestructionDegreeCriterion.CONSTANT)
solution = solver_single.search()
print(f"best iteration {solver_single.stats['best_iteration']} best objective {solver_single.stats['best_objective']}")

Terminating LNS in 198 iterations and obj 489
best iteration 192 best objective 489


In [148]:
solver_parallel = PLNS(
    problem_1,
    initial_solution_1,
    time_limit=200,
    nb_processes=11,
    min_destruction_degree=0.05,
    max_destruction_degree=0.2,
    # destroy_strategy=DestroyStrategy.HIGHEST_COST,
    # repair_strategy=RepairStrategy.GREEDY_LEAST_COST_INSERT,
)
# solver.set_destruction_degree_criterion(DestructionDegreeCriterion.CONSTANT)
solution = solver_parallel.search()
print(f"best iteration {solver_parallel.stats['best_iteration']} best objective {solver_parallel.stats['best_objective']}")

Parallel LNS initialized with 11 processes
Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 103 iterations and obj 467
Parallel LNS terminating w/ objective 467 in 103 iterations in 200.53332233428955 secs
best iteration 56 best objective 467


In [145]:
def plot_solver_stats(solver, fig=None):
    fig = fig or make_subplots(
        rows=1,
        cols=2,
        shared_yaxes=True,
    )

    objectives = [sol.objective for sol in solver.solutions_cache]
    iterations = [sol.iteration for sol in solver.solutions_cache]
    time_found = [
        "{:.2f}".format(sol.time_to_find) for sol in solver.solutions_cache
    ]

    improving_solutions = []
    best_obj = solver.solutions_cache[0].objective
    for i, sol in enumerate(solver.solutions_cache):
        if sol.objective < best_obj:
            best_obj = sol.objective
            improving_solutions.append(sol)

    improving_iterations = [sol.iteration for sol in improving_solutions]
    improving_objectives = [sol.objective for sol in improving_solutions]
    improving_times = [
        "{:.2f}".format(sol.time_to_find) for sol in improving_solutions
    ]

    # fig.update_layout(xaxis2= {'anchor': 'y', 'overlaying': 'x', 'side': 'top'})

    fig.add_trace(go.Scatter(x=iterations,
                            y=objectives,
                            name="objective along iterations",
                            # marker_color="#636EFA"
                            ),
                row=1,
                col=1)
    fig.add_trace(go.Scatter(x=time_found,
                            y=objectives,
                            name="objective along time",
                            # marker_color="#636EFA"
                            ),
                row=1,
                col=2)

    fig.add_trace(go.Scatter(x=improving_iterations,
                            y=improving_objectives,
                            mode='markers',
                            name="improving iterations",
                            # marker_color="green"
                            ),
                row=1,
                col=1)
    fig.add_trace(go.Scatter(x=improving_times,
                            y=improving_objectives,
                            mode='markers',
                            name="improving times",
                            # marker_color="green"
                            ),
                row=1,
                col=2)

    # fig.data[1].update(xaxis='x2')
    # fig.data[3].update(xaxis='x2')

    fig.update_xaxes(title_text="iteration", row=1, col=1)
    fig.update_xaxes(title_text="time", row=1, col=2)
    fig.update_yaxes(title_text="objective", row=1, col=1)

    fig.update_layout(height=600, width=1600, title_text="Objective Value as the LNS Progresses")
    return fig

In [149]:
fig = plot_solver_stats(solver_single)
fig = plot_solver_stats(solver_parallel, fig)

fig.show()

In [164]:
# Try with small problem problem 2
# 15  itmes: default min and max degree (0.15, 0.35) -> (2,5)

time_limit = 100
nb_processes_list = list(range(2,
                               mp.cpu_count()))  # exclusive so n - 1, (1, 11)

single = LNS(problem_2, initial_solution_2, time_limit=time_limit)
solvers = [single] + [
    PLNS(problem_2, initial_solution_2, nb_processes=nb, time_limit=time_limit)
    for nb in nb_processes_list
]

for solver in solvers:
    solver.search()

Parallel LNS initialized with 2 processes
Parallel LNS initialized with 3 processes
Parallel LNS initialized with 4 processes
Parallel LNS initialized with 5 processes
Parallel LNS initialized with 6 processes
Parallel LNS initialized with 7 processes
Parallel LNS initialized with 8 processes
Parallel LNS initialized with 9 processes
Parallel LNS initialized with 10 processes
Parallel LNS initialized with 11 processes
Terminating LNS in 423 iterations and obj 277
Terminating LNS in 437 iterations and obj 268
Parallel LNS terminating w/ objective 268 in 437 iterations in 100.07994890213013 secs
Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 418 iterations and obj 242
Parallel LNS terminating w/ objective 242 in 418 iterations in 100.2078640460968 secs
Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 295 iterations and obj 260
Parallel LNS terminating w/ objective 260 in 295 iterations in 100.08608722686768 secs
Para

In [166]:
# Use LNS with greedy repair
greedy_repair_solver = LNS(
    problem_2,
    initial_solution_2,
    time_limit=time_limit,
    repair_strategy=RepairStrategy.GREEDY_LEAST_COST_INSERT)
greedy_repair_solver.search()


Terminating LNS in 230 iterations and obj 276


Solution(
	order=0, 9, 24, 11, 5, 12, 15, 7, 26, 13, 4, 27, 20, 1, 16, 28, 3, 8, 19, 10, 18, 6, 22, 2, 21, 17, 23, 14, 29, 30, 25, 
	step: 0 vertex: 0 	0, 0, 0, 0, 0, 0
	step: 1 vertex: 9 	400, 0, 0, 0, 0, 0
	step: 2 vertex: 24 	0, 0, 0, 0, 0, 0
	step: 3 vertex: 11 	0, 0, 0, 400, 0, 0
	step: 4 vertex: 5 	0, 0, 0, 400, 400, 0
	step: 5 vertex: 12 	0, 0, 200, 400, 400, 0
	step: 6 vertex: 15 	0, 600, 200, 400, 400, 0
	step: 7 vertex: 7 	200, 600, 200, 400, 400, 0
	step: 8 vertex: 26 	200, 600, 200, 0, 400, 0
	step: 9 vertex: 13 	400, 600, 200, 0, 400, 0
	step: 10 vertex: 4 	400, 600, 200, 0, 400, 400
	step: 11 vertex: 27 	400, 600, 0, 0, 400, 400
	step: 12 vertex: 20 	400, 600, 0, 0, 0, 400
	step: 13 vertex: 1 	800, 600, 0, 0, 0, 400
	step: 14 vertex: 16 	400, 600, 0, 0, 0, 400
	step: 15 vertex: 28 	200, 600, 0, 0, 0, 400
	step: 16 vertex: 3 	400, 600, 0, 0, 0, 400
	step: 17 vertex: 8 	400, 600, 0, 0, 400, 400
	step: 18 vertex: 19 	400, 600, 0, 0, 400, 0
	step: 19 vertex: 10 	400, 600, 600

In [176]:
# Initialize figure with subplots
fig = make_subplots(
    rows=1, cols=2, shared_yaxes=True,
)

solvers_to_plot = [0, 4, 10] # not parallel, 5 processes, 11 processes 
solvers_to_plot = [solvers[i] for i in solvers_to_plot]
solvers_to_plot.append(greedy_repair_solver)

for solver in solvers_to_plot:
    nb_proc = 1
    if isinstance(solver, PLNS):
        nb_proc = solver.nb_processes
    if solver.repair_strategy == RepairStrategy.GREEDY_LEAST_COST_INSERT:
        strategy = "greedy"
    else:
        strategy = "least cost"

    objectives = [sol.objective for sol in solver.solutions_cache]
    iterations = [sol.iteration for sol in solver.solutions_cache]
    time_found = [
        "{:.2f}".format(sol.time_to_find) for sol in solver.solutions_cache
    ]

    improving_solutions = []
    best_obj = solver.solutions_cache[0].objective
    for i, sol in enumerate(solver.solutions_cache):
        if sol.objective < best_obj:
            best_obj = sol.objective
            improving_solutions.append(sol)

    improving_iterations = [sol.iteration for sol in improving_solutions]
    improving_objectives = [sol.objective for sol in improving_solutions]
    improving_times = [
        "{:.2f}".format(sol.time_to_find) for sol in improving_solutions
    ]

    # fig.update_layout(xaxis2= {'anchor': 'y', 'overlaying': 'x', 'side': 'top'})

    fig.add_trace(go.Scatter(x=iterations,
                            y=objectives,
                            name=f"{nb_proc} obj vs iterations {strategy}",
                            # marker_color="#636EFA"
                            ),
                row=1,
                col=1)
    fig.add_trace(go.Scatter(x=time_found,
                            y=objectives,
                            name=f"{nb_proc} obj vs times {strategy} ",
                            # marker_color="#636EFA"
                            ),
                row=1,
                col=2)

    fig.add_trace(go.Scatter(x=improving_iterations,
                            y=improving_objectives,
                            mode='markers',
                            name=f"improving {nb_proc} iterations {strategy}",
                            # marker_color="green"
                            ),
                row=1,
                col=1)
    fig.add_trace(go.Scatter(x=improving_times,
                            y=improving_objectives,
                            mode='markers',
                            name=f"improving {nb_proc} times {strategy}",
                            # marker_color="green"
                            ),
                row=1,
                col=2)


    fig.update_xaxes(title_text="iteration", row=1, col=1)
    fig.update_xaxes(title_text="time", row=1, col=2)
    fig.update_yaxes(title_text="objective", row=1, col=1)

fig.update_layout(height=800, width=1800)
fig.show()

In [189]:
time_limit = 20
destroy_strategies = [DestroyStrategy.RANDOM, DestroyStrategy.HIGHEST_COST]
repair_strategies = [RepairStrategy.GREEDY_LEAST_COST_INSERT, RepairStrategy.RANDOM_SINGLE_ORDER_LEAST_COST]

operator_solvers = []
for des_strategy in destroy_strategies:
    for rep_strategy in repair_strategies:
        solver = LNS(problem_2, initial_solution_2, time_limit=time_limit, destroy_strategy=des_strategy, repair_strategy=rep_strategy)
        operator_solvers.append(solver)

for solver in operator_solvers:
    solver.search()

Terminating LNS in 44 iterations and obj 273
Terminating LNS in 88 iterations and obj 311
Terminating LNS in 36 iterations and obj 434
Terminating LNS in 72 iterations and obj 437


In [183]:
solver.repair_strategy.name

'RANDOM_SINGLE_ORDER_LEAST_COST'

In [190]:
# Initialize figure with subplots
fig = make_subplots(
    rows=1, cols=2, shared_yaxes=True,
)

solvers_to_plot = operator_solvers 
names = [str((solver.destroy_strategy.name, solver.repair_strategy.name)) for solver in solvers_to_plot]

for s, solver in enumerate(solvers_to_plot):

    objectives = [sol.objective for sol in solver.solutions_cache]
    iterations = [sol.iteration for sol in solver.solutions_cache]
    time_found = [
        "{:.2f}".format(sol.time_to_find) for sol in solver.solutions_cache
    ]

    improving_solutions = []
    best_obj = solver.solutions_cache[0].objective
    for i, sol in enumerate(solver.solutions_cache):
        if sol.objective < best_obj:
            best_obj = sol.objective
            improving_solutions.append(sol)

    improving_iterations = [sol.iteration for sol in improving_solutions]
    improving_objectives = [sol.objective for sol in improving_solutions]
    improving_times = [
        "{:.2f}".format(sol.time_to_find) for sol in improving_solutions
    ]

    # fig.update_layout(xaxis2= {'anchor': 'y', 'overlaying': 'x', 'side': 'top'})

    fig.add_trace(go.Scatter(x=iterations,
                            y=objectives,
                            name=f"obj along iterations {names[s]}",
                            # marker_color="#636EFA"
                            ),
                row=1,
                col=1)
    fig.add_trace(go.Scatter(x=time_found,
                            y=objectives,
                            name=f"obj along time {names[s]}",
                            # marker_color="#636EFA"
                            ),
                row=1,
                col=2)

    fig.add_trace(go.Scatter(x=improving_iterations,
                            y=improving_objectives,
                            mode='markers',
                            name=f"improving iterations {names[s]}",
                            # marker_color="green"
                            ),
                row=1,
                col=1)
    fig.add_trace(go.Scatter(x=improving_times,
                            y=improving_objectives,
                            mode='markers',
                            name=f"improving times {names[s]}",
                            # marker_color="green"
                            ),
                row=1,
                col=2)


    fig.update_xaxes(title_text="iteration", row=1, col=1)
    fig.update_xaxes(title_text="time", row=1, col=2)
    fig.update_yaxes(title_text="objective", row=1, col=1)

fig.update_layout(height=800, width=1800)
fig.show()